## Hole filling using gaussian overviews
David Shean  
Updated June 2023

In [ ]:
import os
import sys

In [ ]:
#datadir = '/Users/dshean/Google Drive/My Drive/Chamoli_DEMs/DEM/WV_before/2015_mosaics/release'

In [ ]:
#cd $datadir

In [ ]:
fn = 'Chamoli_Sept2015_Composite_DEM2m_wmean_crop3Dprint.tif'

In [ ]:
fn = '/Users/dshean/Library/CloudStorage/GoogleDrive-dshean@uw.edu/My Drive/DSI_STV_2022/UW_fusion_deeplearning/baker20150911_csm_stereo_raster_stack/try_pc_align_to_lidar_15m_maxdisp_rotationallowed-1.0m-DEM.tif'

In [ ]:
ovr_lvl = [2, 4, 8, 16, 32, 64, 128]
ovr_lvl_str = ' '.join(map(str, ovr_lvl))

In [ ]:
!gdaladdo -r gauss "$fn" $ovr_lvl_str

In [ ]:
out_fn_list = []

for lvl in ovr_lvl:
    p = 100./lvl
    out_fn = os.path.splitext(fn)[0]+f'_{lvl}x.tif'
    !gdal_translate -outsize $p% $p% \
    -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER \
    "$fn" "$out_fn"
    out_fn_list.append(out_fn)

In [ ]:
out_fn_list_str = ' '.join(f"'{w}'" for w in out_fn_list)
out_fn_list_str

In [ ]:
out_fn_list[i]

In [ ]:
mos_fn = os.path.splitext(fn)[0]+f'_gaussfill_multistage.tif'

In [ ]:
temp = fn
for i in range(len(out_fn_list)):
    #blending_px = 2**(i+1)
    blending_px = 2**i
    !dem_mosaic --priority-blending-length $blending_px "$temp" "{out_fn_list[i]}" -o '/tmp/temp.tif'
    temp = '/tmp/temp.tif'
!mv /tmp/temp.tif "$mos_fn"

In [ ]:
blending_px = 3

In [ ]:
mos_fn = os.path.splitext(fn)[0]+f'_fill_{blending_px}px.tif'

In [ ]:
mos_fn

In [ ]:
!dem_mosaic --priority-blending-length $blending_px "$fn" $out_fn_list_str -o "$mos_fn"

In [ ]:
mos_fn = os.path.splitext(fn)[0]+'_fill_wavg.tif'

In [ ]:
!dem_mosaic "$fn" $out_fn_list_str -o "$mos_fn"

In [ ]:
mos_fn = os.path.splitext(fn)[0]+'_fill_first.tif'

In [ ]:
!dem_mosaic --first "$fn" $out_fn_list_str -o "$mos_fn"

In [ ]:
#This actually worked the best for filling DEM to create continuous orthoimages
!gdal_fillnodata.py -si 3 "$fn" "${fn}_gdalfill_si3.tif"